In [201]:
# Imports and paths
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style='whitegrid')

PATH_SNAPSHOT = '/root/The-Beautiful-Game-Oracle/understat_data/EPL/team_elos_v2.csv'
PATH_TS = '/root/The-Beautiful-Game-Oracle/understat_data/EPL/Team_Results/team_elos_timeseries.csv'
PATH_FUTURE = '/root/The-Beautiful-Game-Oracle/understat_data/EPL/future_games_EPL_2025.csv'
assert os.path.exists(PATH_SNAPSHOT), f'Missing snapshot CSV: {PATH_SNAPSHOT}'
assert os.path.exists(PATH_TS), f'Missing timeseries CSV: {PATH_TS}'
assert os.path.exists(PATH_FUTURE), f'Missing future fixtures CSV: {PATH_FUTURE}'
print('Using files:', PATH_SNAPSHOT, 'and', PATH_TS, 'and', PATH_FUTURE)

Using files: /root/The-Beautiful-Game-Oracle/understat_data/EPL/team_elos_v2.csv and /root/The-Beautiful-Game-Oracle/understat_data/EPL/Team_Results/team_elos_timeseries.csv and /root/The-Beautiful-Game-Oracle/understat_data/EPL/future_games_EPL_2025.csv


In [202]:
# Load data
df_snap = pd.read_csv(PATH_SNAPSHOT)
df_ts = pd.read_csv(PATH_TS)
print('Snapshot shape:', df_snap.shape)
print('Timeseries shape:', df_ts.shape)
display(df_snap.head(3))

Snapshot shape: (25, 6)
Timeseries shape: (1250, 13)


,team,final_elo,played,wins,draws,losses
0,Arsenal,1629.99,125,82,27,16
1,Manchester City,1618.07,125,84,21,20
2,Liverpool,1573.20,125,74,29,22


In [203]:
# Helper to pick first available column name

def first_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

# Detect core columns from original timeseries
date_col = first_col(df_ts, ['date','Date','match_date','matchDate'])
order_col = first_col(df_ts, ['matchday','round','game_number','game_index','fixture','fixture_id','match_id']) or date_col

# Try direct team + elo columns
team_direct = first_col(df_ts, ['team','Team','club','Club'])
elo_direct = first_col(df_ts, ['elo','team_elo','elo_after','post_elo','current_elo','rating','after_elo','elo_rating'])

# Try home/away pair columns
home_team_col = first_col(df_ts, ['home_team','HomeTeam','home'])
away_team_col = first_col(df_ts, ['away_team','AwayTeam','away'])
home_elo_post_col = first_col(df_ts, ['home_elo_post','home_rating_post','home_elo_after','home_post_elo'])
away_elo_post_col = first_col(df_ts, ['away_elo_post','away_rating_post','away_elo_after','away_post_elo'])

if team_direct is not None and elo_direct is not None:
    # Use direct team + elo
    df_ts_team = df_ts[[c for c in [team_direct, date_col, order_col, elo_direct] if c is not None]].copy()
    df_ts_team = df_ts_team.rename(columns={team_direct: 'team', elo_direct: 'elo_post'})
elif (home_team_col is not None and away_team_col is not None and
      home_elo_post_col is not None and away_elo_post_col is not None):
    # Build team-level long dataframe from home/away post ELO
    parts = []
    cols_home = [c for c in [date_col, order_col, home_team_col, home_elo_post_col] if c is not None]
    cols_away = [c for c in [date_col, order_col, away_team_col, away_elo_post_col] if c is not None]
    df_home = df_ts[cols_home].copy().rename(columns={home_team_col: 'team', home_elo_post_col: 'elo_post'})
    df_away = df_ts[cols_away].copy().rename(columns={away_team_col: 'team', away_elo_post_col: 'elo_post'})
    parts.extend([df_home, df_away])
    df_ts_team = pd.concat(parts, ignore_index=True)
else:
    raise ValueError('Could not derive team-level ELO from timeseries (no direct team/elo or home/away elo_post columns).')

# Normalize team strings
if df_ts_team['team'].dtype == object:
    df_ts_team['team'] = df_ts_team['team'].astype(str).str.strip()

# Final column names the rest of the notebook will use
team_col = 'team'
elo_col = 'elo_post'

print('Timeseries prepared -> team:', team_col, '| order cand:', order_col, '| date:', date_col, '| elo:', elo_col,
      '| rows:', len(df_ts_team))

Timeseries prepared -> team: team | order cand: match_id | date: date | elo: elo_post | rows: 2500


In [204]:
# Compute last-5 ELO deltas per team

# Work from the team-level timeseries built above
df_ts_proc = df_ts_team.copy()

# Parse date if available and not numeric
date_col_eff = None
if 'date' in df_ts_proc.columns or (date_col is not None and date_col in df_ts_proc.columns):
    col = date_col if date_col in df_ts_proc.columns else 'date'
    if not np.issubdtype(df_ts_proc[col].dtype, np.number):
        df_ts_proc[col] = pd.to_datetime(df_ts_proc[col], errors='coerce')
    date_col_eff = col

# Choose effective ordering column
order_col_eff = None
if order_col is not None and order_col in df_ts_proc.columns:
    order_col_eff = order_col
elif date_col_eff is not None:
    order_col_eff = date_col_eff
else:
    df_ts_proc['_ord_'] = np.arange(len(df_ts_proc))
    order_col_eff = '_ord_'

# Sort and compute within-team diffs
df_ts_proc = df_ts_proc.sort_values(by=[team_col, order_col_eff])
df_ts_proc['elo_delta'] = df_ts_proc.groupby(team_col)[elo_col].diff()

# Build a mapping of team -> last 5 deltas (most recent first)
last5_map = {}
for t, sub in df_ts_proc.groupby(team_col, sort=False):
    deltas = sub['elo_delta'].dropna().tail(5).tolist()
    deltas = deltas[::-1]  # most recent first
    while len(deltas) < 5:
        deltas.append(np.nan)
    last5_map[t] = deltas

len(last5_map)

25

In [205]:
# Compute current ELO per team and map next opponent from fixtures

# Current ELO per team (latest by order/date)
team_order_eff = None
if 'matchday' in df_ts_team.columns or 'round' in df_ts_team.columns or 'fixture' in df_ts_team.columns or 'match_id' in df_ts_team.columns:
    # prefer any present ordering column used earlier
    for cand in ['matchday','round','fixture','match_id']:
        if cand in df_ts_team.columns:
            team_order_eff = cand
            break
elif 'date' in df_ts_team.columns:
    team_order_eff = 'date'
else:
    df_ts_team['_ord_cur_'] = np.arange(len(df_ts_team))
    team_order_eff = '_ord_cur_'

cur_sorted = df_ts_team.sort_values([team_col, team_order_eff])
current_elo_map = cur_sorted.groupby(team_col)[elo_col].last().to_dict()

# Fallback: snapshot final ELO
snap_team_col_fallback = 'team' if 'team' in df_snap.columns else ( 'Team' if 'Team' in df_snap.columns else None )
snap_final_elo_map = {}
if snap_team_col_fallback is not None and 'final_elo' in df_snap.columns:
    snap_final_elo_map = dict(zip(df_snap[snap_team_col_fallback].astype(str).str.strip(), df_snap['final_elo']))

# Load future fixtures and build next-game mapping per team
df_future = pd.read_csv(PATH_FUTURE)

home_f = first_col(df_future, ['home_team','HomeTeam','home'])
away_f = first_col(df_future, ['away_team','AwayTeam','away'])
date_f = first_col(df_future, ['date','Date','match_date','matchDate','kickoff','kickoff_time'])

if home_f is None or away_f is None:
    raise ValueError('Could not locate home/away team columns in future fixtures.')

# Normalize
for c in [home_f, away_f]:
    if df_future[c].dtype == object:
        df_future[c] = df_future[c].astype(str).str.strip()

if date_f is not None and not np.issubdtype(df_future[date_f].dtype, np.number):
    df_future[date_f] = pd.to_datetime(df_future[date_f], errors='coerce')

# Define season window (Aug 2025 - May 2026) and compute participating teams
SEASON_START = pd.Timestamp(2025, 8, 1)
SEASON_END = pd.Timestamp(2026, 5, 31, 23, 59, 59)
if date_f is not None:
    df_future_season = df_future[(df_future[date_f] >= SEASON_START) & (df_future[date_f] <= SEASON_END)].copy()
else:
    df_future_season = df_future.copy()

teams_in_season = set(pd.unique(pd.concat([df_future_season[home_f], df_future_season[away_f]], ignore_index=True)))

# Reference date = latest date we have in the timeseries (or now if missing)
ref_date = None
if 'date' in df_ts_team.columns:
    ref_date = pd.to_datetime(df_ts_team['date'], errors='coerce').max()
if pd.isna(ref_date) or ref_date is None:
    ref_date = pd.Timestamp.utcnow()

next_date_map = {}
next_opp_map = {}
next_opp_elo_map = {}

# Sort fixtures by date if available to pick earliest future
if date_f is not None:
    df_future = df_future.sort_values(date_f)

for t in teams_in_season:
    sub = df_future[(df_future[home_f] == t) | (df_future[away_f] == t)]
    if date_f is not None:
        sub = sub[sub[date_f] >= ref_date]
        if sub.empty:
            continue
        row = sub.iloc[0]
    else:
        if sub.empty:
            continue
        row = sub.iloc[0]

    opp = row[away_f] if row[home_f] == t else row[home_f]
    opp_norm = str(opp).strip()
    next_opp_map[t] = opp_norm
    next_date_map[t] = row[date_f] if date_f is not None else pd.NaT
    # Opponent ELO: prefer current map, fallback to snapshot
    opp_elo = current_elo_map.get(opp_norm, np.nan)
    if pd.isna(opp_elo):
        opp_elo = snap_final_elo_map.get(opp_norm, np.nan)
    next_opp_elo_map[t] = opp_elo

len(current_elo_map), len(next_opp_map), len(teams_in_season)

(25, 20, 20)

In [206]:
# Build league table from snapshot and join last-5 deltas + next game info

snap_team_col = first_col(df_snap, ['team','Team']) or 'team'
cols_keep = [c for c in [snap_team_col,'played','wins','draws','losses'] if c in df_snap.columns]
table = df_snap[cols_keep].copy()

# Normalize team names for joining (without changing display column)
if table[snap_team_col].dtype == object:
    table['_team_norm'] = table[snap_team_col].astype(str).str.strip()
else:
    table['_team_norm'] = table[snap_team_col].astype(str)

# Filter to teams in the current season window if available
try:
    table = table[table['_team_norm'].isin(teams_in_season)].copy()
except Exception:
    pass

# Attach last 5
L5_cols = ['L5_1','L5_2','L5_3','L5_4','L5_5']
for c in L5_cols:
    table[c] = np.nan

for idx, row in table.iterrows():
    t = row['_team_norm']
    if t in last5_map:
        vals = last5_map[t]
        for i, c in enumerate(L5_cols):
            table.at[idx, c] = vals[i]

# Attach current ELO, next opponent, next date, opponent ELO and diff
# current ELO fallback to snapshot final elo when missing
table['elo'] = table['_team_norm'].map(current_elo_map).fillna(table.get('final_elo', np.nan))
table['next_opponent'] = table['_team_norm'].map(next_opp_map)
table['next_date'] = table['_team_norm'].map(next_date_map)
table['opp_elo'] = table['_team_norm'].map(next_opp_elo_map)
table['elo_diff'] = table['elo'] - table['opp_elo']

# Preferred column order: L5 before next-opponent fields (no points)
order_pref = [
    snap_team_col,
    'played','wins','draws','losses',
    'elo',
    'L5_1','L5_2','L5_3','L5_4','L5_5',
    'next_opponent','next_date','opp_elo','elo_diff'
]
cols_existing = [c for c in order_pref if c in table.columns]
# Include any remaining columns (e.g., final_elo) at the end
cols_existing += [c for c in table.columns if c not in cols_existing]
table = table[cols_existing]

# Rank by final_elo desc if present
if 'final_elo' in table.columns:
    table = table.sort_values('final_elo', ascending=False)

# Optional: drop helper column from view
table = table.drop(columns=['_team_norm'])

table.shape

(20, 15)

In [207]:
# Style and display

def color_delta(val):
    if pd.isna(val):
        return 'color: #999999'
    if val > 0:
        return 'color: #2ca02c'  # green
    if val < 0:
        return 'color: #d62728'  # red
    return 'color: #999999'

def fmt_signed(x):
    if pd.isna(x):
        return ''
    return f"{x:+.2f}"


def fmt_date(x):
    if pd.isna(x):
        return ''
    try:
        return pd.to_datetime(x).date().isoformat()
    except Exception:
        return str(x)

fmt = {c: fmt_signed for c in ['L5_1','L5_2','L5_3','L5_4','L5_5','elo_diff']}
for c in ['elo','opp_elo']:
    if c in table.columns:
        fmt[c] = '{:.2f}'.format
if 'next_date' in table.columns:
    fmt['next_date'] = fmt_date

# Sort for display
table = table.sort_values('elo', ascending=False)

# Build styler with improved aesthetics (used by dataframe_image export)
header_bg = "#0035a0"
cell_border = '1px solid #e6e6e6'
caption = f"EPL ELO Table 2025–26 — generated {pd.Timestamp.utcnow().date().isoformat()}"

styler = (
    table
      .style
      .hide(axis='index')
      .set_caption(caption)
      .applymap(color_delta, subset=['L5_1','L5_2','L5_3','L5_4','L5_5'])
      .applymap(color_delta, subset=['elo_diff'])
      .set_table_styles([
          {'selector': 'th', 'props': [('background-color', header_bg), ('font-weight', 'bold'), ('border-bottom', cell_border)]},
          {'selector': 'td', 'props': [('border', cell_border), ('padding', '6px 8px')]},
          {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('font-size','13px')]},
          {'selector': 'caption', 'props': [('caption-side','top'), ('font-size','14px'), ('font-weight','600'), ('margin-bottom','6px')]},
      ])
      .set_properties(subset=[c for c in ['team','next_opponent'] if c in table.columns], **{'text-align': 'left'})
      .set_properties(subset=[c for c in table.columns if c not in ['team','next_opponent']], **{'text-align': 'right'})
      .format(fmt)
)
styler

/tmp/ipykernel_21512/1622466817.py:42: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  table


team,played,wins,draws,losses,elo,L5_1,L5_2,L5_3,L5_4,L5_5,next_opponent,next_date,opp_elo,elo_diff
Arsenal,125,82,27,16,1629.99,-1.42,+10.60,+5.03,+6.84,+4.95,Tottenham,,1511.96,+118.03
Manchester City,125,84,21,20,1618.07,+13.36,+8.99,-8.46,+7.49,+7.35,Newcastle United,,1513.74,+104.33
Liverpool,125,74,29,22,1573.20,-13.36,+9.84,-8.56,-11.06,-7.94,Nottingham Forest,,1465.75,+107.45
Chelsea,125,55,31,39,1568.19,+7.04,+9.02,-10.23,+15.94,+7.93,Burnley,,1453.93,+114.26
Aston Villa,125,62,27,36,1551.68,+16.50,-9.84,+8.46,+8.93,+4.95,Leeds,,1456.44,+95.24
Crystal Palace,125,41,41,43,1543.34,-2.40,+10.45,-5.03,-2.11,-7.53,Wolverhampton Wanderers,,1402.97,+140.37
Brighton,125,50,37,38,1533.50,+2.40,+12.03,-12.27,+7.16,-0.70,Brentford,,1520.92,+12.58
Brentford,125,46,32,47,1520.92,+12.02,-10.45,+8.56,+13.20,-7.35,Brighton,,1533.50,-12.58
Sunderland,11,5,4,2,1520.28,+1.42,-3.08,+10.23,+8.17,-11.83,Fulham,,1472.52,+47.76
Bournemouth,125,44,29,52,1517.16,-16.51,-8.99,+8.08,+2.11,+9.16,West Ham,,1447.62,+69.54


In [208]:
# Export league table as an image (PNG)
import os
import math
import matplotlib.pyplot as plt

# Output path
EXPORT_DIR = '/root/The-Beautiful-Game-Oracle/understat_data/EPL/outputs'
os.makedirs(EXPORT_DIR, exist_ok=True)
IMG_PATH = os.path.join(EXPORT_DIR, 'elo_league_table_2025_26.png')

# Try high-fidelity export with dataframe_image; fallback to Matplotlib table
use_dfi = False
try:
    import dataframe_image as dfi  # pip install dataframe-image
    use_dfi = True
except Exception:
    use_dfi = False

if use_dfi:
    try:
        dfi.export(styler, IMG_PATH)
        print('Saved image via dataframe_image to:', IMG_PATH)
    except Exception as e:
        print('dataframe_image export failed, falling back to Matplotlib:', e)
        use_dfi = False

if not use_dfi:
    # Build a compact view for Matplotlib table export
    # Choose a sensible team display column from the current table
    team_display_col = 'team' if 'team' in table.columns else ('Team' if 'Team' in table.columns else table.columns[1])

    display_cols_pref = [
        'rank' if 'rank' in table.columns else None,
        team_display_col,
        'played' if 'played' in table.columns else None,
        'wins' if 'wins' in table.columns else None,
        'draws' if 'draws' in table.columns else None,
        'losses' if 'losses' in table.columns else None,
        # removed points column
        'elo' if 'elo' in table.columns else None,
        # L5 goes BEFORE next-opponent fields
        'L5_1' if 'L5_1' in table.columns else None,
        'L5_2' if 'L5_2' in table.columns else None,
        'L5_3' if 'L5_3' in table.columns else None,
        'L5_4' if 'L5_4' in table.columns else None,
        'L5_5' if 'L5_5' in table.columns else None,
        # Next-opponent group
        'next_opponent' if 'next_opponent' in table.columns else None,
        'next_date' if 'next_date' in table.columns else None,
        'opp_elo' if 'opp_elo' in table.columns else None,
        'elo_diff' if 'elo_diff' in table.columns else None,
    ]
    display_cols = [c for c in display_cols_pref if c is not None]

    view = table.copy()
    if 'next_date' in view.columns:
        view['next_date'] = pd.to_datetime(view['next_date'], errors='coerce').dt.date.astype('string').fillna('')
    # Prepare formatted strings for numeric columns, but keep raw values for color logic later
    for c in ['final_elo','elo','opp_elo']:
        if c in view.columns:
            view[c] = view[c].map(lambda x: '' if pd.isna(x) else f"{x:.2f}")
    for c in ['elo_diff','L5_1','L5_2','L5_3','L5_4','L5_5']:
        if c in view.columns:
            view[c] = view[c].map(lambda x: '' if pd.isna(x) else f"{x:+.2f}")

    data = view[display_cols]

    nrows = len(data)
    ncols = len(display_cols)

    # Layout heuristics (tighter): smaller row height, controlled width, minimal margins
    row_h = 0.30
    # Per-column width tuning
    col_widths = []
    for name in display_cols:
        if name in ['rank']:
            col_widths.append(0.50)
        elif name in [team_display_col, 'next_opponent']:
            col_widths.append(1.6)
        elif name in ['next_date']:
            col_widths.append(1.1)
        else:
            col_widths.append(0.9)

    fig_w = max(9, sum(col_widths) * 0.8)
    fig_h = max(3.0, nrows * row_h + 1.0)

    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    ax.axis('off')
    plt.subplots_adjust(left=0.02, right=0.98, top=0.92, bottom=0.06)

    # Title & footnote
    title = f"EPL ELO Table 2025–26"
    sub = f"Generated {pd.Timestamp.utcnow().strftime('%Y-%m-%d %H:%M UTC')}"
    fig.suptitle(title, fontsize=14, fontweight='bold', y=0.98)
    fig.text(0.02, 0.94, sub, fontsize=9, color='#666666')

    tbl = ax.table(
        cellText=data.values,
        colLabels=[c.replace('_', ' ').title() for c in display_cols],
        loc='upper left',  # avoid wide left margin
        cellLoc='right',   # right-align numbers by default
        colWidths=[w / sum(col_widths) for w in col_widths],
    )

    tbl.auto_set_font_size(False)
    tbl.set_fontsize(9)
    # Tighten overall row height
    tbl.scale(1.0, 0.85)

    # Header formatting and smaller header height
    header_bg = '#f4f5f7'
    edge = '#dddddd'
    for j in range(ncols):
        cell = tbl[0, j]
        cell.set_facecolor(header_bg)
        cell.get_text().set_fontweight('bold')
        cell.set_height(0.28)
        cell.set_edgecolor(edge)
        cell.set_linewidth(0.6)

    # Column-specific alignment: left-align team and next_opponent
    left_align_cols = [name for name in [team_display_col, 'next_opponent'] if name in display_cols]
    col_index = {name: idx for idx, name in enumerate(display_cols)}
    for name in left_align_cols:
        j = col_index[name]
        # Header text alignment
        tbl[0, j].get_text().set_ha('left')

    # Row zebra striping + borders
    for i in range(nrows):
        trow = i + 1  # data rows start at 1
        row_bg = '#fbfbfd' if i % 2 else '#ffffff'
        for j in range(ncols):
            cell = tbl[trow, j]
            cell.set_facecolor(row_bg)
            cell.set_edgecolor(edge)
            cell.set_linewidth(0.4)
            # Text align per column
            if display_cols[j] in left_align_cols:
                cell.get_text().set_ha('left')
            else:
                cell.get_text().set_ha('right')

    # Subtle group shading for L5 and Next-opponent columns
    l5_cols = [name for name in ['L5_1','L5_2','L5_3','L5_4','L5_5'] if name in display_cols]
    next_cols = [name for name in ['next_opponent','next_date','opp_elo','elo_diff'] if name in display_cols]
    for name in l5_cols:
        j = col_index[name]
        # Header shade
        tbl[0, j].set_facecolor('#f7fbf7')
    for name in next_cols:
        j = col_index[name]
        tbl[0, j].set_facecolor('#f7f7fb')

    # Color rules for L5 deltas, elo_diff, and elo vs opp_elo
    for i in range(nrows):
        trow = i + 1
        # L5 columns
        for name in l5_cols:
            j = col_index[name]
            raw = table.iloc[i][name] if name in table.columns else np.nan
            txt = tbl[trow, j].get_text()
            if pd.isna(raw):
                txt.set_color('#999999')
            elif raw > 0:
                txt.set_color('#2ca02c')
            elif raw < 0:
                txt.set_color('#d62728')
            else:
                txt.set_color('#999999')
        # elo_diff
        if 'elo_diff' in col_index and 'elo_diff' in table.columns:
            j = col_index['elo_diff']
            raw = table.iloc[i]['elo_diff']
            txt = tbl[trow, j].get_text()
            if pd.isna(raw):
                txt.set_color('#999999')
            elif raw > 0:
                txt.set_color('#2ca02c')
            elif raw < 0:
                txt.set_color('#d62728')
            else:
                txt.set_color('#999999')
        # elo vs opp_elo pair
        if 'elo' in col_index and 'opp_elo' in col_index and 'elo' in table.columns and 'opp_elo' in table.columns:
            j_cur = col_index['elo']
            j_opp = col_index['opp_elo']
            cur_raw = pd.to_numeric(table.iloc[i]['elo'], errors='coerce')
            opp_raw = pd.to_numeric(table.iloc[i]['opp_elo'], errors='coerce')
            cur_txt = tbl[trow, j_cur].get_text()
            opp_txt = tbl[trow, j_opp].get_text()
            if not pd.isna(cur_raw) and not pd.isna(opp_raw):
                if cur_raw > opp_raw:
                    cur_txt.set_color('#2ca02c')
                    opp_txt.set_color('#d62728')
                elif cur_raw < opp_raw:
                    cur_txt.set_color('#d62728')
                    opp_txt.set_color('#2ca02c')
                else:
                    cur_txt.set_color('#999999')
                    opp_txt.set_color('#999999')

    fig.savefig(IMG_PATH, dpi=240, bbox_inches='tight', pad_inches=0.05)
    plt.close(fig)
    print('Saved image via Matplotlib to:', IMG_PATH)

IMG_PATH

dataframe_image export failed, falling back to Matplotlib: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.
Saved image via Matplotlib to: /root/The-Beautiful-Game-Oracle/understat_data/EPL/outputs/elo_league_table_2025_26.png
Saved image via Matplotlib to: /root/The-Beautiful-Game-Oracle/understat_data/EPL/outputs/elo_league_table_2025_26.png


'/root/The-Beautiful-Game-Oracle/understat_data/EPL/outputs/elo_league_table_2025_26.png'

In [209]:
# Export league table as CSV
import os

EXPORT_DIR = '/root/The-Beautiful-Game-Oracle/understat_data/EPL/outputs'
os.makedirs(EXPORT_DIR, exist_ok=True)
CSV_PATH = os.path.join(EXPORT_DIR, 'elo_league_table_2025_26.csv')

csv_df = table.copy()
if 'next_date' in csv_df.columns:
    csv_df['next_date'] = pd.to_datetime(csv_df['next_date'], errors='coerce').dt.date.astype('string').fillna('')

# Ensure ELO-related columns are true floats so float_format applies
num_cols = [c for c in ['elo','opp_elo','elo_diff','L5_1','L5_2','L5_3','L5_4','L5_5'] if c in csv_df.columns]
for c in num_cols:
    csv_df[c] = pd.to_numeric(csv_df[c], errors='coerce')

# Write with fixed-point precision to avoid scientific notation / 3-sig-fig display
csv_df.to_csv(CSV_PATH, index=False, float_format='%.2f')
print('Saved CSV to:', CSV_PATH)
CSV_PATH

Saved CSV to: /root/The-Beautiful-Game-Oracle/understat_data/EPL/outputs/elo_league_table_2025_26.csv


'/root/The-Beautiful-Game-Oracle/understat_data/EPL/outputs/elo_league_table_2025_26.csv'